In [ ]:
import transformers
from transformers import pipeline
import pandas as pd
import re
import spacy

In [ ]:
#carico il modello per l'italiano
!python -m spacy download it_core_news_lg
nlp = spacy.load("it_core_news_lg")

In [ ]:
#imposto il modello BERT da usare con la libreria pipeline
unmasker = pipeline('fill-mask', model="dbmdz/bert-base-italian-xxl-cased", tokenizer = "dbmdz/bert-base-italian-xxl-cased")

Prova su 8 frasi rappresentative (1 MASK)

In [ ]:
#creo una variabile con le frasi di prova
sentences = ["1) Mi piace bere il vino. Quando bevo [MASK] mi sento bene.",
             "2) Mi piace bere l'acqua. Quando bevo [MASK] mi sento bene.",
             "3) Mi piace bere succhi di frutta. Quando bevo [MASK] mi sento bene.",
             "4) Mi piace bere i succhi di frutta, l'acqua e il vino. Quando bevo [MASK] mi sento bene",
             "5) Versò la minestra e gliela diede da mangiare. Lui mangiò [MASK] e ringraziò",
             "6) Versò la minestra e gliela diede da bere. Lui bevve [MASK] e ringraziò",
             "7) Versò la pozione e gliela diede da bere. Lui bevve [MASK] e ringraziò",
             "8) Versò la pozione verde e gliela diede da bere. Lui bevve [MASK] verde e ringraziò"]

In [ ]:
for sentence in sentences:
    noun_predictions = []
    #uso l'unmasker per ottenere i primi 10 risultati
    results = unmasker(sentence, top_k=10)
    #per ogni risultato, applico il pos tagging di SpaCy in modo da isolare solo i nomi
    for prediction in results:
        doc = nlp(prediction["token_str"])
        #aggiungo i nomi candidati alla lista noun_predictions
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        #ordino i candidati in base al prediction score
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        #isolo il candidato con lo score più alto e stampo il punteggio e la frase completa
        print(noun_predictions[0]["sequence"])
    else:
        print("unknown")

Prova su 80 frasi dell'IMPLICIT dataset, primi 4 verbi (2 MASK)

In [ ]:
#importo le frasi dal dataset
file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/IMPLICIT_results.xlsx')
sentences = file['frase'][:80]
#for i in sentences:
  #print(i)

Prova 0:
1.   fornire una lista di Determiners
2.   iterare su questi per completare il Noun
3.   prendere la combinazione con punteggio più alto

In [ ]:
output = []

for sentence in sentences:
    noun_predictions = [] #1
    det_predictions = ["il ", "lo ", "la ", "i ", "gli ", "le ", ""]
    for det in det_predictions:
        new_sentence = sentence.replace("[MASK]", det, 1)
        results = unmasker(new_sentence, top_k=10)
        for prediction in results: #2
            doc = nlp(prediction["token_str"])
            for token in doc:
                if token.pos_ == "NOUN":
                    noun_predictions.append(prediction)
    if noun_predictions: #3
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output.append(noun_predictions[0]["token_str"])
        #print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], noun_predictions[0]["sequence"])
    else:
        output.append("unknown")
        #print("unknown")
#print(output)

prova 1:
1.   riempire il primo [MASK] con un Determiner
2.   creare una nuova frase con il primo [MASK] completato
3.   completare la nuova frase con il Noun con il punteggio più alto

In [ ]:
output1 = []
for sentence in sentences:
    results = unmasker(sentence, top_k=10)
    det_predictions = [] #1
    for prediction in results[0]:
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "DET":
                det_predictions.append(prediction)
    if det_predictions:
            det_predictions.sort(key=lambda x: x["score"], reverse=True)
            determiner = det_predictions[0]["token_str"]
            new_sentence = sentence.replace("[MASK]", determiner, 1) #2
    else:
            new_sentence = sentence.replace("[MASK]", "", 1)
    results_new = unmasker(new_sentence, top_k=10) #3
    noun_predictions = []
    for prediction in results_new:
        doc = nlp(prediction['token_str'])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output1.append(noun_predictions[0]["token_str"])
        #print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], noun_predictions[0]["sequence"])
    else:
        output1.append("unknown")
        #print("unknown")
#print(output1)

prova 2:
1.   riempire il secondo [MASK] ignorando il primo



In [ ]:
output2 = []
for sentence in sentences:
    results = unmasker(sentence, top_k=10)
    noun_predictions = [] #1
    for prediction in results[1]:
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "NOUN":
                noun_predictions.append(prediction)
    if noun_predictions:
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        output2.append(noun_predictions[0]["token_str"])
        sequence = noun_predictions[0]["sequence"]
        sequence = re.sub("\[CLS\]","", sequence)
        sequence = re.sub("\[SEP\]","", sequence)
        #print(noun_predictions[0]["token_str"], noun_predictions[0]["score"], sequence)
    else:
        output2.append("unknown")
        #print("unknown")
#print(output2)

Secondo step -- creazione di un **dizionario** e **calcolo** **della** **similarity**

In [ ]:
#creo tre dizionari che contengono i risultati dei tre esperimenti

#carico la lista di annotazioni manuali per le prime 80 frasi
predicted = file['obj_1'][:80]

#il dizionario associa il risultato del modello all'annotazione manuale creando delle coppie
word_dict = {}
counter = {}
for predicted_word, output_word in zip(predicted, output):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        #per evitare sovrapposizioni, aggiungo una numerazione alle parole ripetute
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict[new_key] = output_word
#print("word_dict: ", word_dict)
#print(len(word_dict))

#ripeto il passaggio sui risultati del secondo esperimento
word_dict1 = {}
counter = {}
for predicted_word, output_word in zip(predicted, output1):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict1[new_key] = output_word
#print("word_dict1: ", word_dict1)
#print(len(word_dict1))

#ripeto il passaggio sui risultati del terzo esperimento
word_dict2 = {}
counter = {}
for predicted_word, output_word in zip(predicted, output2):
    if predicted_word not in word_dict:
        word_dict[predicted_word] = output_word
    else:
        if predicted_word in counter:
            counter[predicted_word] += 1
        else:
            counter[predicted_word] = 1
        new_key = f"{predicted_word}_{counter[predicted_word]}"
        word_dict2[new_key] = output_word
#print("word_dict2: ", word_dict2)
#print(len(word_dict2))

In [ ]:
#calcolo la media di similarity tra le parole dei tre dizionari e l'annotazione manuale

#definisco la funzione per calcolare la similarity tra due parole
def calculate_similarity(word1, word2):
    #rimuovo sistematicamente il numero di indice aggiunto prima
    word1 = re.sub(r"_\d+", "", word1)
    word2 = re.sub(r"_\d+", "", word2)
    word1_token = nlp(word1)
    word2_token = nlp(word2)
    if word1 == "unknown" or word2 == "unknown":
        return 0
    if word1_token.has_vector and word2_token.has_vector:
        similarity = word1_token.similarity(word2_token)
        return similarity
    else:
      return 0

similarity_scores = []
for key, value in word_dict.items():
    similarity = calculate_similarity(key, value)
    similarity_scores.append(similarity)
#print(similarity_scores)
print("media esperimento 0 =", sum(similarity_scores) / len(similarity_scores))

similarity_scores1 = []
for key, value in word_dict1.items():
    similarity = calculate_similarity(key, value)
    similarity_scores1.append(similarity)
#print(similarity_scores1)
print("media esperimento 1 =", sum(similarity_scores1) / len(similarity_scores1))

similarity_scores2 = []
for key, value in word_dict2.items():
    similarity = calculate_similarity(key, value)
    similarity_scores2.append(similarity)
#print(similarity_scores2)
print("media esperimento 2 =", sum(similarity_scores2) / len(similarity_scores2))